In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import plotly.express as px
import plotly.io as pio
from scipy import stats
pio.renderers.default = "vscode"
pio.renderers
from plotly.graph_objs import *
from song_characteristics import *
from plots import *
from spot_secrets import *
import requests
import urllib.request

In [44]:
user_id = "fd3d877b-6d54-4cba-a037-ed9ab548f8af"

In [53]:
conn, cursor = db_connect("spotify_db.db")
sp = connect_to_sp(cid, secret)
dataframe = download_data(user_id, cursor)

In [ ]:
counts = dataframe.groupby("song_name")["song_name"].size().reset_index(name="count")

In [ ]:
dataframe["sec_played"] = dataframe["ms_played"]/1000

In [ ]:
dataframe = dataframe.drop(["ms_played"], axis=1)
dataframe = dataframe.groupby("song_name")["song_name"].size().reset_index(name="count")
dataframe = dataframe.groupby('count').size().reset_index(name='number_of_songs')
plot_df = dataframe.copy()
fig = px.scatter(x=plot_df["number_of_songs"], y=plot_df["count"], labels={"x":"Number of songs", "y":"Number of plays"}, color_discrete_sequence=['#1DB954'])
fig.update_layout(
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    title_font_color="white",
    title = {
        "text":"Do you attach to songs?",
        "x": 0.5,
        "xanchor":"center",
        "yanchor": "top"
    }
)
fig.update_xaxes(color="white")
fig.update_yaxes(color="white")

In [ ]:
dataframe = download_data(user_id, cursor)

In [ ]:
total_listening_time, favorite_artists, favorite_songs, distinct_artists, distinct_songs, favorite_artists_minutes, favorite_artist_fraction, favorite_songs_of_fav_artist, number_of_songs_by_fav_artist, favorite_morning, favorite_evening = get_general_statistics(dataframe)

In [ ]:
favorite_songs_of_fav_artist

In [ ]:
favorite_artists

In [ ]:
dataframe["min_played"] = dataframe["ms_played"]/60000

In [ ]:
total_listening_time = dataframe["min_played"].sum() # Total listening time
favorite_artists = dataframe.groupby("artist_name")["artist_name"].size().reset_index(name="count").sort_values("count", ascending=False) # Favorite artists
favorite_songs = dataframe.groupby(["artist_name", "song_name"]).size().reset_index(name="count").sort_values("count", ascending=False) # Favorite songs
distinct_artists = len(dataframe.drop_duplicates("artist_name")) # Distinct artists
distinct_songs = len(dataframe.drop_duplicates("song_name")) # Distinct songs


In [ ]:
# Stats for favorite artist
favorite_artists_minutes = dataframe.groupby("artist_name")["min_played"].sum().astype(int).reset_index(name="sum_of_minutes").sort_values("sum_of_minutes", ascending=False) # Minutes of favorite artists
favorite_artist_fraction = favorite_artists_minutes["sum_of_minutes"].iloc[0] / favorite_artists_minutes["sum_of_minutes"][1:].sum() # Percentage of minutes occupied by favorite artist
favorite_songs_of_fav_artist = favorite_songs[favorite_songs["artist_name"] == favorite_artists["artist_name"].iloc[0]] # Favorite songs of favorite artist
number_of_songs_by_fav_artist = len(favorite_songs_of_fav_artist) # Number of distinct songs of favorite artist listened

In [ ]:
dataframe["hour"] = pd.to_datetime(dataframe["end_time"]).dt.hour
dataframe_evening = dataframe[(dataframe["hour"].astype(int) > 19) | (dataframe["hour"].astype(int) < 5)]
dataframe_morning = dataframe[(dataframe["hour"].astype(int) > 5) & (dataframe["hour"].astype(int) < 10)]

In [ ]:
favorite_evening = dataframe_evening.groupby(["artist_name", "song_name"]).size().reset_index(name="count").sort_values("count", ascending=False) # Favorite morning songs
favorite_morning = dataframe_morning.groupby(["artist_name", "song_name"]).size().reset_index(name="count").sort_values("count", ascending=False) # Favorite evening songs

In [ ]:
favorite_morning.head()

In [ ]:
dataframe['date'] = pd.to_datetime(dataframe['end_time'])

In [ ]:
dataframe['date'] = pd.to_datetime(dataframe['date'].dt.date)

In [ ]:
dataframe['hour'] = pd.to_datetime(dataframe['end_time']).dt.hour

In [ ]:
dataframe['week'] = pd.to_datetime(dataframe['end_time']).dt.week

In [ ]:
df_grouped = dataframe.groupby(['week', 'hour']).size().reset_index(name='count')

In [ ]:
dataframe[(dataframe["week"] == 48) & (dataframe["hour"] == 0)]

In [ ]:
df_grouped[df_grouped["week"] == 48]

In [ ]:
make_general_heatmap(dataframe)

In [54]:
personal_data = get_data_with_statistics(user_id, cursor)
recommender_data = get_data_from_recommender(cursor)

In [5]:
personal_data.head()

,artist_name,song_name,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,end_time
0,Playboi Carti,Shoota (feat. Lil Uzi Vert),0.674,0.617,-9.680,0.2220,0.1850,0.000,0.1060,0.451,153.103,2022-05-14 09:18
1,Playboi Carti,Fell In Luv (feat. Bryson Tiller),0.657,0.668,-6.208,0.1360,0.0273,0.000,0.3200,0.227,162.997,2022-05-14 09:18
2,Playboi Carti,Magnolia,0.791,0.582,-7.323,0.2860,0.0114,0.000,0.3500,0.443,162.991,2022-05-14 09:21
3,Trippie Redd,Miss The Rage (feat. Playboi Carti),0.581,0.877,-4.938,0.1140,0.0151,0.000,0.8260,0.353,77.006,2022-05-14 17:34
4,070 Shake,Terminal B,0.703,0.393,-11.191,0.0374,0.0755,0.819,0.0876,0.153,135.009,2022-05-14 17:40


In [55]:
recommender_results = recommend_me(personal_data, recommender_data, ["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"])

0       0.743905
1       0.850776
2       0.816455
3       0.889867
4       0.697396
          ...   
1298    0.788476
1299    0.735071
1300    0.751231
1301    0.567810
1302    0.559684
Name: loudness, Length: 1303, dtype: float64


In [57]:
recommender_results

{'artist_personal': ['Miły ATZ',
  'Floating Points',
  'Airelle Besson',
  'Charli XCX',
  'Leon Vynehall'],
 'song_personal': ['Day Off',
  'Movement 5',
  'The Sound of Your Voice, Pt. II',
  'Selfish Girl',
  'Movements (Chapter III)'],
 'artist_database': ['jonatan leandoer96',
  'Sleep Party People',
  'Ross from Friends',
  'The Temper Trap',
  'R.A.P. Ferreira'],
 'song_database': ['Vår Sång',
  'Everything Has an End',
  'A Brand New Start',
  'Sweet Disposition',
  'CYCLES'],
 'distance': [1.0255009115625726,
  0.44983759629795655,
  0.42817657592525227,
  0.3884818924487179,
  0.3525853953993009]}

In [58]:
def get_artists_photos(results_dict, sp):
    artist_ids = []

    for i in range(len(results_dict)):
        artist_personal_name = results_dict["artist_personal"][i]
        artist_recommender_name = results_dict["artist_database"][i]
        result_personal = sp.search(q=artist_personal_name, type='artist')
        artist_id_personal = result_personal['artists']['items'][0]['id']
        result_recommender = sp.search(q=artist_recommender_name, type='artist')
        artist_id_recommender = result_recommender['artists']['items'][0]['id']
        artist_search_personal = sp.artist(artist_id_personal)
        artist_search_recommender = sp.artist(artist_id_recommender)
        artist_image_url_personal = artist_search_personal['images'][0]['url']
        artist_image_url_recommender = artist_search_recommender['images'][0]['url']

        urllib.request.urlretrieve(artist_image_url_personal, "../website/static/artist_image_personal" + str(i) + ".jpg")
        urllib.request.urlretrieve(artist_image_url_recommender, "../website/static/artist_image_recommender" + str(i) + ".jpg")


    

In [59]:
get_artists_photos(recommender_results, sp)